In [ ]:
from datetime import datetime as dt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as st

from sklearn.preprocessing import StandardScaler

### データ読み込み

In [ ]:
train_kaggle = pd.read_csv('train_kaggle.csv')
train_kaggle.datetime = pd.to_datetime(train_kaggle.datetime)
train_kaggle.datetime.min(), train_kaggle.datetime.max()

 ### 目的変数をドルに変換

In [ ]:
minimum_unit = train_kaggle.operation_value.abs().min()
train_kaggle['target'] = train_kaggle.operation_value / minimum_unit
train_kaggle['target'] = train_kaggle['target'].astype(int)
train_kaggle.head()

### ヒストグラム

In [ ]:
print(train_kaggle.target.mean(),train_kaggle.target.min(),train_kaggle.target.max(),)
train_kaggle.target.hist(bins=30);

In [ ]:
for atm_id in by_atm.keys():
    idx = by_atm[atm_id].datetime
    x = by_atm[atm_id].target
    plt.plot(idx, x);



### ヒートマップ

In [ ]:
train_kaggle['dayofweek'] = train_kaggle.datetime.dt.dayofweek
train_kaggle['day'] = train_kaggle.datetime.dt.day
train_kaggle['month'] = train_kaggle.datetime.dt.month
train_kaggle['week'] = train_kaggle.datetime.dt.week
sns.heatmap(train_kaggle.pivot_table(index='day', columns='dayofweek', values='target'));

### ATM 別のトランザクション数

In [ ]:

print(len(train_kaggle.atm_id.unique()))
train_kaggle.atm_id.value_counts()[[ 74,  85,  86,  87,  88,  90,  91,  93,  94,  96,  97,  98,  99,
            100, 101, 102, 103, 104, 105, 106]].plot(kind='barh', figsize=(10, 10));

### 前処理

In [ ]:
def read_weather_data(path):
    """
    pathのデフォルトは 'dailydata.csv'
    """
    weather_data = pd.read_csv(path, encoding='shift_jis')
    weather_data.datetime = pd.to_datetime(weather_data.datetime)
    weather_data = weather_data.drop(columns=['年', '月', '日'])
    return weather_data

def merge_weather_data(data, weather_data):
    return pd.merge(data, weather_data, how='inner', on='datetime')
    
    

def sumrize_daily_data(data):
        daily_data = data.pivot_table(
                                index=['datetime', 'atm_id', 'month', 'dayofweek', 'day'], 
                                values='target',
                                aggfunc='sum').reset_index()
        return daily_data

def add_poly_feature(data, target_feature, dim):
    """
    多項式回帰用: 指定したfeatureを次数dimまでの特徴量を追加する
    """
    if dim < 2:
        return data
    else:
        poly_feature = np.vstack([data[target_feature]**i for i in range(1, dim)]).T
        feature_names = [f'{target_feature}^{i}' for i in range(2, dim)]
        for i, feature_name in enumerate(feature_names):
            data[feature_name] = poly_feature[:, i]
        return data

def add_passed_date_feature(data, datefrom):
    """
    datefrom からの経過日数特徴量を追加する
    フォーマッt : YYYY-mm-dd
    """
    datefrom = dt.strptime(datefrom, '%Y-%m-%d')
    return (data.datetime - datefrom).dt.days

def add_salary10th(data):
    return data['day'].replace({10: 1, 1: 0, 2: 0, 3: 0, 4: 0
                                                   , 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 11: 0
                                                   , 12: 0, 13: 0, 14: 0, 15: 0, 16: 0
                                                   , 17: 0, 18: 0, 19: 0, 20: 0, 21: 0
                                                   , 22: 0, 23: 0, 24: 0, 25: 0, 26: 0
                                                   , 27: 0, 28: 0, 29: 0, 30: 0, 31: 0})

def add_salary25th(data):
    return data['day'].replace({25: 1, 1: 0, 2: 0, 3: 0, 4: 0
                                                   , 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 11: 0
                                                   , 12: 0, 13: 0, 14: 0, 15: 0, 16: 0
                                                   , 17: 0, 18: 0, 19: 0, 20: 0, 21: 0
                                                   , 22: 0, 23: 0, 24: 0, 10: 0, 26: 0
                                                   , 27: 0, 28: 0, 29: 0, 30: 0, 31: 0})


def preprocess(data):
    daily_data = sumrize_daily_data(data)
    weather_data = read_weather_data('dailydata.csv')
    daily_data = merge_weather_data(daily_data, weather_data)
    
    daily_data['passed_date'] = add_passed_date_feature(daily_data, '2017-06-09')
    # , :
    for col in ['passed_date', 'day', '日平均気温(℃)', '日最高気温(℃)', '日最低気温(℃)', '日降水量(mm)']:
        daily_data = add_poly_feature(daily_data, col, 5)
        
    daily_data['salary10th'] = add_salary10th(daily_data)
    daily_data['salary25th'] = add_salary25th(daily_data)    
    daily_data = pd.get_dummies(data=daily_data, columns=['dayofweek', 'month'], drop_first=True)
    return daily_data


def get_feature_target_cols(data):
    
    target_col = 'target'
    exclude_cols = ['target', 'datetime', 'client_id', 'operation_value', 'atm_id', 'operation_type']
    feature_cols = [col for col in data.columns if col not in exclude_cols]  # data_preprocessed には前処理ずみのデータを入れる
    
    return feature_cols, target_col


def split_by_atm(data):
    dataset_by_atm = {}
    for atm_id in data.atm_id.unique(): 
        is_target = atm_id == data.atm_id
        data_by_atm = data[is_target].reset_index(drop=True) 
        dataset_by_atm[atm_id] = data_by_atm
    return dataset_by_atm


def read_atm_data(path):
    data = pd.read_csv(path)
    data.datetime = pd.to_datetime(data.datetime)
    return data


def to_dollar(data):    
    minimum_unit = data.operation_value.abs().min()
    dollar = data.operation_value / minimum_unit
    dollar = dollar.astype(int)
    return dollar


def add_date_features(data):
    data['dayofweek'] = data.datetime.dt.dayofweek
    data['day'] = data.datetime.dt.day
    data['month'] = data.datetime.dt.month
#     data['week'] = data.datetime.dt.week
    return data

def data_preparation_main(path='train_kaggle.csv'):
    data = read_atm_data(path)
    data['target'] = to_dollar(data)
    data = add_date_features(data)
    preprocessed = preprocess(data)
    by_atm = split_by_atm(preprocessed)
    feature_cols, target_col = get_feature_target_cols(preprocessed)
    return preprocessed, by_atm, feature_cols,target_col

### pymc

In [ ]:
import scipy.stats as st

import pymc3 as pm
import arviz as az

import warnings
warnings.simplefilter('ignore')

In [ ]:
by_atm[87].shape

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

from sklearn.tree import DecisionTreeRegressor

train = by_atm[87]
X = train[feature_cols].values
y = train[target_col].values

lr = Lasso(alpha=1)
lr.fit(X, y)
plt.hist(lr.coef_);

In [ ]:
def get_date_bucket(x):
    if x < 10:
        return 'month_head'
    elif x < 20:
        return 'month_mid'
    else:
        return 'month_end'

In [ ]:
feature_for_sd = [
 'passed_date', 
 'dayofweek_1',
 'dayofweek_2',
 'dayofweek_3',
 'dayofweek_4',
 'dayofweek_5',
 'dayofweek_6',
 'month_2',
 'month_3',
 'month_4',
 'month_5',
 'month_6',
 'month_7',
 'month_8',
 'month_9',
 'month_10',
 'month_11',
 'month_12',
 'salary10th',
 'salary25th',
 'month_head',
 'month_mid',
 'month_end']

In [ ]:
train = by_atm[87]
X = train[feature_cols].values
y = train[target_col].values

train['month_head'] = (train['day'] <= 10).astype(int)
train['month_mid'] = ((10 < train['day'])&(train['day'] <= 20)).astype(int)
train['month_end'] = (20 < train['day']).astype(int)    

X_for_sd = train[feature_for_sd].values

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_for_sd = scaler.fit_transform(X_for_sd)

X = scaler.fit_transform(X)

In [ ]:
N = train['passed_date'].max()
date = train['passed_date'].values

In [ ]:
with pm.Model() as model:    
    # 回帰係数
    mu_w_1 = pm.Uniform('mu_w_1', lower=-1000, upper=1000, shape=len(feature_cols))
    mu_w_2 = pm.Uniform('mu_w_2', lower=-1000, upper=1000, shape=len(feature_cols))
    
    # 切片
    mu_intercept_1 = pm.Uniform('mu_intercept_1', lower=-1000, upper=1000,)
    mu_intercept_2 = pm.Uniform('mu_intercept_2', lower=-1000, upper=1000,)
    
    # y = α + βX
    mu_1 = pm.math.dot(X, mu_w_1) + mu_intercept_1
    mu_2 = pm.math.dot(X, mu_w_2) + mu_intercept_2
    
    #  y の変化点
    mu_tau = pm.DiscreteUniform('mu_tau', lower=0, upper=N)
    mu = pm.Deterministic('mu', pm.math.switch(mu_tau > date, mu_1, mu_2))
        
    # 分散の重み
    sd_w_1 = pm.Uniform('sd_w_1', lower=-1000, upper=1000, shape=len(feature_for_sd))
    sd_w_2 = pm.Uniform('sd_w_2', lower=-1000, upper=1000, shape=len(feature_for_sd))
    sd_intercept_1 = pm.Uniform('sd_intercept_1', lower=-1000, upper=1000)
    sd_intercept_2 = pm.Uniform('sd_intercept_2', lower=-1000, upper=1000)
    
    # 分散を線形結合で求める
    sd_1 = pm.math.dot(X_for_sd, sd_w_1) + sd_intercept_1
    sd_2 = pm.math.dot(X_for_sd, sd_w_2) + sd_intercept_2
    
    #  sdの変化点
    sd_tau = pm.DiscreteUniform('sd_tau', lower=0, upper=N)
    sd = pm.Deterministic('sd', pm.math.switch(sd_tau > date, sd_1, sd_2))
    
    pm_y = pm.Normal('pm_y', mu=mu, sd=pm.math.exp(sd), observed=y)

In [ ]:
with model:
    trace = pm.sample(draws=1000, chains=1, step=pm.Metropolis())

In [ ]:
trace.varnames

In [ ]:
az.summary(trace, var_names=['mu_tau', 'sd_tau'])

In [ ]:
coef_result = az.summary(trace, var_names=['mu_w_1', 'mu_w_2'])
coef_result['feature_name'] = feature_cols * 2
coef_result

In [ ]:
az.summary(trace, var_names=['sd_w_1', 'sd_w_2'])

In [ ]:
az.plot_trace(trace, var_names=['mu_tau', 'sd_tau']);

In [ ]:
sd_w_dataframe = pd.DataFrame(index=feature_for_sd, data={'sd_before': trace['sd_w_1'].mean(axis=0), 'sd_after':trace['sd_w_2'].mean(axis=0)})
sd_w_dataframe.plot(kind='barh', figsize=(20, 10));

In [ ]:
coef_w_dataframe = pd.DataFrame(index=feature_cols, data={'coef_before': trace['mu_w_1'].mean(axis=0), 'coef_after':trace['mu_w_2'].mean(axis=0)})
coef_w_dataframe.plot(kind='barh', figsize=(20, 10));


In [ ]:
sd_tau_matrix = np.ones([len(date), len(trace['sd_tau'])])
sd_tau = ((sd_tau_matrix * trace['sd_tau']).T > date).mean(axis=0)

sd_1 = X_for_sd @ trace['sd_w_1'].mean(axis=0) + trace['sd_intercept_1'].mean()
sd_2 = X_for_sd @ trace['sd_w_2'].mean(axis=0) + trace['sd_intercept_2'].mean()

sd_before = sd_1 * sd_tau
sd_after = sd_2 * (1 - sd_tau)

sd = sd_before + sd_after
# sd

In [ ]:
mu_tau_matrix = np.ones([len(date), len(trace['mu_tau'])])
mu_tau = ((mu_tau_matrix * trace['mu_tau']).T > date).mean(axis=0)

mu_1 = X @ trace['mu_w_1'].mean(axis=0) + trace['mu_intercept_1'].mean()
mu_2 = X @ trace['mu_w_2'].mean(axis=0) + trace['mu_intercept_2'].mean()

mu_before = mu_1 * mu_tau
mu_after = mu_2 * (1 - mu_tau)

mu = mu_before + mu_after
# mu

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(train.datetime, train.target)
plt.plot(train.datetime, mu)

plt.fill_between(train.datetime, mu + (1.96 * np.exp(sd)), mu - (1.96 * np.exp(sd)), alpha=0.4);